### Following code is a slightly modified code, that was provided by Hauke Bartsch from the following link: https://github.com/mmiv-center/Research-Information-System/blob/master/components/Workflow-Image-AI/src/templates/python/stub.py

In [2]:
import pydicom
import glob
import numpy as np
import sys
import os
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

### Remember to comment out which datafolders you're not using, according to your machine

The following folders are the paths for the alienware-ubuntu machine at MMIV 

In [3]:
def which_computer_folder(which_computer: str):
    if which_computer == "home":
        datafolder = "C:/Users/Heris/OneDrive/Dokumenter/data/"
        outputfolder = "C:/Users/Heris/OneDrive/Dokumenter/data/OUTPUT/"
    elif which_computer == "mmiv":
        datafolder = "/home/heris/data/LIDC-IDRI/"
        outputfolder = "/home/heris/data/OUTPUT/LIDC-IDRI/"
    elif which_computer != "home" or which_computer != "mmiv":
        raise ValueError
    return datafolder, outputfolder

In [4]:
datafolder , outputfolder = which_computer_folder("mmiv")
# datafolder , outputfolder = which_computer_folder("home")

In [5]:
metadata = pd.read_csv(datafolder+"metadata.csv")
columns = metadata.columns
metadata = metadata[metadata["Number of Images"] >= 10]
metadata["Modality"]


,Series UID,Collection,3rd Party Analysis,Data Description URI,Subject ID,Study UID,Study Description,Study Date,Series Description,Manufacturer,Modality,SOP Class Name,SOP Class UID,Number of Images,File Size,File Location,Download Timestamp
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.179049373636...,LIDC-IDRI,NaN,https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX,LIDC-IDRI-0001,1.3.6.1.4.1.14519.5.2.1.6279.6001.298806137288...,NaN,01-01-2000,NaN,GE MEDICAL SYSTEMS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,133,70.33 MB,./LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-30...,2023-10-05T16:22:59.579
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.170706757615...,LIDC-IDRI,NaN,https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX,LIDC-IDRI-0003,1.3.6.1.4.1.14519.5.2.1.6279.6001.101370605276...,NaN,01-01-2000,NaN,GE MEDICAL SYSTEMS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,140,74.11 MB,./LIDC-IDRI/LIDC-IDRI-0003/01-01-2000-NA-NA-94...,2023-10-05T16:23:05.032
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.619372068417...,LIDC-IDRI,NaN,https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX,LIDC-IDRI-0002,1.3.6.1.4.1.14519.5.2.1.6279.6001.490157381160...,NaN,01-01-2000,NaN,GE MEDICAL SYSTEMS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,261,137.73 MB,./LIDC-IDRI/LIDC-IDRI-0002/01-01-2000-NA-NA-98...,2023-10-05T16:23:09.037
8,1.3.6.1.4.1.14519.5.2.1.6279.6001.129007566048...,LIDC-IDRI,NaN,https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX,LIDC-IDRI-0005,1.3.6.1.4.1.14519.5.2.1.6279.6001.190188259083...,NaN,01-01-2000,NaN,GE MEDICAL SYSTEMS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,133,70.08 MB,./LIDC-IDRI/LIDC-IDRI-0005/01-01-2000-NA-NA-42...,2023-10-05T16:23:29.224
9,1.3.6.1.4.1.14519.5.2.1.6279.6001.323541312620...,LIDC-IDRI,NaN,https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX,LIDC-IDRI-0004,1.3.6.1.4.1.14519.5.2.1.6279.6001.191425307197...,NaN,01-01-2000,NaN,GE MEDICAL SYSTEMS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,241,126.93 MB,./LIDC-IDRI/LIDC-IDRI-0004/01-01-2000-NA-NA-91...,2023-10-05T16:23:32.578
12,1.3.6.1.4.1.14519.5.2.1.6279.6001.132817748896...,LIDC-IDRI,NaN,https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX,LIDC-IDRI-0006,1.3.6.1.4.1.14519.5.2.1.6279.6001.324680252006...,NaN,01-01-2000,NaN,GE MEDICAL SYSTEMS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,133,70.09 MB,./LIDC-IDRI/LIDC-IDRI-0006/01-01-2000-NA-NA-92...,2023-10-05T16:23:45.264
14,1.3.6.1.4.1.14519.5.2.1.6279.6001.272348349298...,LIDC-IDRI,NaN,https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX,LIDC-IDRI-0007,1.3.6.1.4.1.14519.5.2.1.6279.6001.280315210397...,NaN,01-01-2000,NaN,GE MEDICAL SYSTEMS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,145,76.84 MB,./LIDC-IDRI/LIDC-IDRI-0007/01-01-2000-NA-NA-81...,2023-10-05T16:24:03.691
15,1.3.6.1.4.1.14519.5.2.1.6279.6001.774060103415...,LIDC-IDRI,NaN,https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX,LIDC-IDRI-0008,1.3.6.1.4.1.14519.5.2.1.6279.6001.185810436275...,NaN,01-01-2000,NaN,GE MEDICAL SYSTEMS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,133,70.07 MB,./LIDC-IDRI/LIDC-IDRI-0008/01-01-2000-NA-NA-30...,2023-10-05T16:24:05.828
19,1.3.6.1.4.1.14519.5.2.1.6279.6001.286061375572...,LIDC-IDRI,NaN,https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX,LIDC-IDRI-0009,1.3.6.1.4.1.14519.5.2.1.6279.6001.225213110794...,NaN,01-01-2000,NaN,GE MEDICAL SYSTEMS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,256,134.79 MB,./LIDC-IDRI/LIDC-IDRI-0009/01-01-2000-NA-NA-07...,2023-10-05T16:24:37.828
20,1.3.6.1.4.1.14519.5.2.1.6279.6001.416701701108...,LIDC-IDRI,NaN,https://doi.org/10.7937/K9/TCIA.2015.LO9QL9SX,LIDC-IDRI-0010,1.3.6.1.4.1.14519.5.2.1.6279.6001.303099231937...,NaN,01-01-2000,NaN,GE MEDICAL SYSTEMS,CT,CT Image Storage,1.2.840.10008.5.1.4.1.1.2,277,145.90 MB,./LIDC-IDRI/LIDC-IDRI-0010/01-01-2000-NA-NA-43...,2023-10-05T16:24:39.383


In [103]:
if not(os.path.exists(outputfolder)):
    try:
        os.mkdir(outputfolder,0o777)
    except OSError as error:
        (error)

files = []
lidc_idri_00xx = []
for root, dirs, the_files in os.walk(datafolder):
    lidc_idri_00xx = []
    # len(dirs) = 0 tells us that we're in the final folder, and 
    # len(the_files) >= 10 tells us that we only want to look at folders with more than 10 images, 
    # CT is usually only the ones that have that many images 
    """ print(root, dirs, the_files) """
    """ path = root + "/" + the_files """
    """ if os.path.isdir(path):
        continue """
    
    if len(dirs) == 0 and len(the_files) >= 10:
        lidc_idri_00xx = [] #Put every dicom files associated with one person in seperate lists, because the code below this cell if for a single volume
        for dicomfile in the_files: 
            path = root+"/"+dicomfile
            if os.path.isfile(path):
                try: 
                    dicom = pydicom.dcmread(path)
                    dicom.Modality
                    lidc_idri_00xx.append(dicom)
                except Exception as e: 
                    continue 
    if len(lidc_idri_00xx) > 0:
        files.append(lidc_idri_00xx) #NB: Its a list with the dicomfiles that gets appended
        
    

In [109]:
# make sure we only keep data that has the same shape as the first slice
for i in range(len(files)):
    files[i] = [a for a in files[i] if a.get("PixelData") != None and a.pixel_array.shape == files[i][0].pixel_array.shape]


# make sure we sort the slices by SliceLocation or, if that does not exist by InstanceNumber
def sortFunc(s):
    if "SliceLocation" in s:
        return s.SliceLocation
    else:
        if "InstanceNumber" in s:
            return s.InstanceNumber
        return 0

sup_slices = [] #supremum, the list above

for i in range(len(files)):
    slices = sorted(files[i], key=sortFunc) #slices contains many dcm files, and is a list as well 
    sup_slices.append(slices)



In [114]:
import nibabel as nib 

In [118]:

    

for sub_slices in sup_slices: #sub_slices = slices in this section in the original code at github 
    # pixel aspects, assuming all slices are the same
    ps = sub_slices[0].get("PixelSpacing", [1,1])
    ss = sub_slices[0].get("SliceThickness",1)
    ax_aspect = ps[1]/ps[0]
    sag_aspect = ps[1]/ss
    cor_aspect = ps[0]/ss

    # create 3D array
    img_shape = list(sub_slices[0].pixel_array.shape)
    img_shape.append(len(sub_slices))
    img3d = np.zeros(img_shape)

    

    # fill 3D array with the images from the files
    for i, s in enumerate(sub_slices):
        img2d = s.pixel_array
        img3d[:, :, i] = img2d

    # Creating and saving files as nii
    nifti_image = nib.Nifti1Image(img3d, affine = np.eye(4))
    patient_path = outputfolder+"/"+str(sub_slices[0].PatientID)
    os.mkdir(patient_path)
    nib.save(nifti_image,patient_path+"/"+str(sub_slices[0].PatientID))

    #The commented out code is for plotting

    """ # plot 3 orthogonal slices
    fig=plt.figure(figsize=(6,6))
    fig.patch.set_facecolor('gray')
    gs1 = gridspec.GridSpec(2,2)
    gs1.update(wspace=0.025, hspace=0.05)

    a1 = plt.subplot(gs1[0])
    plt.imshow(img3d[:, :, img_shape[2]//2], cmap='gray')
    a1.set_aspect(ax_aspect)
    a1.set_xticklabels([])

    a2 = plt.subplot(gs1[1])
    plt.imshow(img3d[:, img_shape[1]//2, :], cmap='gray')
    a2.set_aspect(sag_aspect)
    a2.set_xticklabels([])
    a2.set_yticklabels([])

    a3 = plt.subplot(gs1[2])
    plt.imshow(img3d[img_shape[0]//2, :, :], cmap='gray')
    a3.set_aspect(cor_aspect)
    a3.set_yticklabels([])

    plt.show() """
